# Validation report

This notebook is used to prototype the script that generates the validation report for a given date.

The idea is that takes the path to some data of a test, process it, runs the models and generates the validation information and visualization to generate the report.

The report is located in the [docs](docs/validation) folder

The indexes: `start_idx`, `end_idx` and `span` are used to define the range of the data to be used for the validation. Make sure to define them properly together with the sample time.

To export the `YYYYMMDD_MED.csv` data complementary to the `YYYYMMDD_solarMED.csv` data the notebook [test_data_to_csv.ipynb](/home/jmserrano/Nextcloud/Juanmi_MED_PSA/Python) is used.


> [!warning] Report will get overtwritten
> In the last step of this notebook, the report is saved in the `docs` folder, if the report already exists, it will be overwritten.

In [1]:
from pathlib import Path
import os
import hjson
import numpy as np
import pandas as pd
from IPython.display import display
from loguru import logger
import time
import datetime
import json

# Visualization
from phd_visualizations.test_timeseries import experimental_results_plot
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

#
from solarMED_modeling.data_validation import within_range_or_nan_or_max, within_range_or_zero_or_max
from solarMED_modeling.utils import data_conditioning, data_preprocessing


# Setup environment for running MATLAB code if not done externally
# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

# auto reload modules
%load_ext autoreload

# Paths definition
data_path: Path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/data')
config_path: Path = Path(f'{os.getenv("HOME")}/development_psa/models_psa/data')
output_path: Path = Path(f'{os.getenv("HOME")}/development_psa/models_psa/docs/validation')

test_date_str = "20230628"

# Data filenames
filename_process_data = f'{test_date_str}_solarMED.csv'
filename_process_data2 = f'{test_date_str}_MED.csv'

# Resample figures using plotly_resampler
resample_figures = False

sample_rate = '30s'
sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

## Pre-processing

In [2]:
data_paths = [data_path / filename_process_data, data_path / filename_process_data2]

# Load variables information
with open( config_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

df = data_preprocessing(paths=data_paths, sample_rate_key=sample_rate, vars_config=vars_config)

2024-05-09 14:11:26.585 | DEBUG    | solarMED_modeling.utils:process_dataframe:68 - Index([], dtype='object')
2024-05-09 14:11:26.586 | INFO     | solarMED_modeling.utils:process_dataframe:75 - Number of duplicate index values in df: 0
2024-05-09 14:11:26.816 | DEBUG    | solarMED_modeling.utils:process_dataframe:68 - Index([], dtype='object')
2024-05-09 14:11:26.817 | INFO     | solarMED_modeling.utils:process_dataframe:75 - Number of duplicate index values in df: 0
2024-05-09 14:11:27.218 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:55 - Signal id FT-SF-003 not found in dataframe columns.
2024-05-09 14:11:27.226 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tamb to C from C
2024-05-09 14:11:27.226 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_in to C from C
2024-05-09 14:11:27.227 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_in to C from C
2024-05-09 14:11:27.228 |

### Add / modify variables

In [3]:
df = data_conditioning(df, cost_e=cost_e, cost_w=cost_w, sample_rate_numeric=sample_rate_numeric)

2024-05-09 14:11:33.260 | INFO     | solarMED_modeling.utils:data_conditioning:214 - Heat exchanger secondary flow rate estimated successfully.
2024-05-09 14:11:33.306 | INFO     | solarMED_modeling.utils:data_conditioning:244 - Thermal storage discharge flow rate estimated successfully (qts_dis/q3wv_src).
2024-05-09 14:11:33.309 | INFO     | solarMED_modeling.utils:data_conditioning:254 - Auxiliary variables calculated successfully.
2024-05-09 14:11:34.618 | INFO     | solarMED_modeling.utils:data_conditioning:261 - Solar field power calculated successfully.
2024-05-09 14:11:37.616 | INFO     | solarMED_modeling.utils:data_conditioning:268 - Thermal storage power calculated successfully.
/home/jmserrano/development_psa/models_psa/solarMED_modeling/utils/__init__.py:272: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. T

In [4]:
# Make sure the output folders exist
# - output_path/YYYYMMDD
# - output_path/YYYYMMDD/attachments

date_str = df.index[0].strftime("%Y%m%d")

output_folder = output_path / date_str
attachments_folder = output_folder / 'attachments'

output_folder.mkdir(parents=True, exist_ok=True)
attachments_folder.mkdir(parents=True, exist_ok=True)

## Visualize test data


In [5]:
# Generate visualization

# Load plot configuration
with open( config_path / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
# )

# Save figure
save_figure(
    figure_name=f"{date_str}_solarMED_visualization", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'),
    width=fig.layout.width, height=fig.layout.height, scale=2
)

2024-05-09 14:11:47.023 | INFO     | phd_visualizations.test_timeseries:experimental_results_plot:378 - Optimization updates not shown in plot, show_optimization_updates: false
2024-05-09 14:11:47.024 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tamb
2024-05-09 14:11:47.024 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>amb</sub>
2024-05-09 14:11:47.037 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>amb</sub> added in yaxis1 (left), row 1, uncertainty: False, comparison: False
2024-05-09 14:11:47.042 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tmed_c_in
2024-05-09 14:11:47.042 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>med,c,in</sub> (⁰C)
2024-05-09 14:11:47.047 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>m

## Complete system

Better to start from the complete system, and use its parameters to validate the individual components.

In [6]:
# Evaluate model

from solarMED_modeling.solar_med import SolarMED

idx_start = int(round(700 / sample_rate_numeric, 0)) # 10 minutes
span = int(round(700 / sample_rate_numeric, 0)) # 10 minutes
idx_end = len(df)
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    sample_time=sample_rate_numeric,
    curve_fits_path='data/curve_fits.json',
    resolution_mode='simple',
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    msf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    cost_w = 3, # €/m³ 
    cost_e = 0.05 # €/kWhe
)


2024-05-09 14:11:55.966 | INFO     | solarMED_modeling.solar_med:init_matlab_engine:701 - MATLAB engine initialized
2024-05-09 14:11:55.968 | INFO     | solarMED_modeling.solar_med:model_post_init:522 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: False
            - Resolution mode: simple
            - Sample time: 30.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
        


In [67]:

# Run model
# %autoreload 2

for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qhx_s'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} seconds. Current operation state is {model.operating_state.name}")

    df_mod = model.to_dataframe(df_mod, rename_flows=True)
    

In [68]:
# Generate visualization

# Load plot configuration
with open( config_path / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)

# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx<idx_end-1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
# )

# Save figure
save_figure(
    figure_name=f"{date_str}_SolarMED_validation", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

## Thermal storage

In [69]:
# Evaluate model

from solarMED_modeling.thermal_storage import thermal_storage_two_tanks_model

df_ts = df.copy()

# # Define idx_start as the first index non nan for:
# idxs_start = [df_ts.index.get_loc( df_ts['Tts_h_t'].first_valid_index()),
#               df_ts.index.get_loc( df_ts['Tts_h_out'].first_valid_index()),
#               df_ts.index.get_loc( df_ts['Tts_c_b_in'].first_valid_index())]
# idx_start = np.max(idxs_start)
# 
# # Remove all nan for Tts_h_t
# df_ts = df_ts[df_ts['Tts_h_t'].notna()]
# df_ts = df_ts[df_ts['Tts_c_b_in'].notna()]

idx_start = 0
idx_end = len(df_ts)

# idx_start = 0

Th_labels = ['Tts_h_t', 'Tts_h_m', 'Tts_h_b']
Tc_labels = ['Tts_c_t', 'Tts_c_m', 'Tts_c_b']

N = len(Th_labels)
    
df_mod = pd.DataFrame()

Tts_h = np.zeros((idx_end-idx_start+1, N), dtype=float)
Tts_c = np.zeros((idx_end-idx_start+1, N), dtype=float)

Tts_h[0] = np.array( [df_ts.iloc[idx_start][T] for T in Th_labels] )
Tts_c[0] = np.array( [df_ts.iloc[idx_start][T] for T in Tc_labels] )
for idx in range(idx_start, idx_end):
    
    j = idx-idx_start+1
    ds = df_ts.iloc[idx]
        
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
    
    Tts_h[j], Tts_c[j] = thermal_storage_two_tanks_model(
        Ti_ant_h=Tts_h[j-1], Ti_ant_c=Tts_c[j-1],  # [ºC], [ºC]
        Tt_in=ds["Tts_h_in"],  # ºC
        Tb_in=ds["Tts_c_b_in"],  # ºC
        Tamb=ds["Tamb"],  # ºC

        qsrc=ds["qts_src"],  # m³/h
        qdis=ds["qts_dis"],  # m³/h

        UA_h=model.UAts_h,  # W/K
        UA_c=model.UAts_c,  # W/K
        Vi_h=model.Vts_h,  # m³
        Vi_c=model.Vts_c,  # m³
        ts=sample_rate_numeric, Tmin=60  # seg, ºC
    )
    
    out = {label: Tts_h[j][i] for i, label in enumerate(Th_labels)}
    out.update({label: Tts_c[j][i] for i, label in enumerate(Tc_labels)})
    
    result = pd.DataFrame(out, index=[0])
    
    logger.info(f"Finished iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} s")
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)

In [70]:
# Generate visualization

# Load plot configuration
with open(config_path / 'plt_config_thermal_storage.json') as f:
    plt_config = json.load(f)
    
# Sync model index with measured data
df_mod.index = df_ts.index[idx_start:idx if idx < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config, df_ts, df_comp=[df_mod], vars_config=vars_config, resample=resample_figures)

# Save figure
save_figure(
    figure_name=f"{date_str}_thermal_storage_validation", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'),
    width=fig.layout.width, height=fig.layout.height, scale=2
)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'{date_str}_thermal_storage_validation')
# )

## Solar field

In [71]:
# Evaluate model

from solarMED_modeling.solar_field import solar_field_model

idx_start = int(round(700 / sample_rate_numeric, 0)) # 10 minutes
span = int(round(700 / sample_rate_numeric, 0)) # 10 minutes

if span > idx_start:
    logger.warning(f'Span {span} cant be greater than idx_start {idx_start}. Setting span to idx_start')
    span = idx_start

# Initialize result vectors
# q_sf_mod   = np.zeros(len(df), dtype=float)
Tsf_out_mod_delay   = np.zeros(len(df)-idx_start, dtype=float)
# l   = np.zeros(len(df), dtype=float)


# Evaluate model
# Tsf_out_mod[0:idx_start] = df.iloc[0:idx_start]['Tsf_out']
# Tsf_out_mod_delay[0:idx_start] = df.iloc[0:idx_start]['Tsf_out']

Tsf_out_mod_delay[0] = df.iloc[idx_start]['Tsf_out']
for i in range(idx_start+1, len(df)):
    # solar_field_model(Tin: conHotTemperatureType, q: PositiveFloat, I: PositiveFloat, Tamb: float,
    #                   Tout_ant: float, q_ant: np.ndarray[float] = None,
    #                   beta: float = 0.0975, H: float = 2.2, nt=1, np=7 * 5, ns=2, Lt=1.15 * 20, Acs:float= 7.85e-5,
    #                   sample_time=1) -> float:
    ds = df.iloc[i]
    j = i-idx_start
    
    
    start_time = time.time()
    
    Tsf_out_mod_delay[j] = solar_field_model(
        Tin=df.iloc[i-span:i]['Tsf_in'].values, # From current value, up to idx_start samples before 
        q=df.iloc[i:i-span:-1]['qsf'].values, # From current value, up to idx_start samples before, in reverse order
        I=ds['I'], Tamb=ds['Tamb'],  Tout_ant=Tsf_out_mod_delay[j-1],
        sample_time=sample_rate_numeric, consider_transport_delay=True,
        # Model parameters
        # beta=1.1578e-2, H=3.1260, gamma=0.0471
        beta=model.beta_sf, H=model.H_sf, gamma=model.gamma_sf
    )
    
    logger.info(f"Iteration {i} / {len(df)}. Elapsed time: {time.time()-start_time:.2f} s")
    
df_mod = pd.DataFrame(Tsf_out_mod_delay, columns=['Tsf_out'], index=df.index[idx_start:])

In [72]:
# Generate visualization

# Load plot configuration
with open(config_path / 'plt_config_solar_field.json') as f:
    plt_config = json.load(f)

# Sync model index with measured data
df_mod.index = df.index[idx_start:i if i < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=[df_mod], vars_config=vars_config, resample=resample_figures)

# Save figure
save_figure(
    figure_name=f"{date_str}_solar_field_validation", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'),
    width=fig.layout.width, height=fig.layout.height, scale=2
)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'{date_str}_solar_field_validation')
# )

In [73]:
# Evaluate model (inverse)

from solarMED_modeling.solar_field import solar_field_inverse_model

df_on = df.where(df['Tsf_out']-df['Tsf_in']>4).dropna()

idx_start = int(round(700 / sample_rate_numeric, 0)) # 10 minutes
span = int(round(700 / sample_rate_numeric, 0)) # 10 minutes

if span > idx_start:
    logger.warning(f'Span {span} cant be greater than idx_start {idx_start}. Setting span to idx_start')
    span = idx_start
    
# Initialize result vectors
# q_sf_mod   = np.zeros(len(df), dtype=float)
qsf_mod   = np.zeros(len(df_on)-idx_start+span, dtype=float)
# l   = np.zeros(len(df), dtype=float)

# Evaluate model
qsf_mod[0:span] = df_on.iloc[idx_start-span:idx_start]['qsf']
# qsf_mod[0:idx_start] = df_on.iloc[0:idx_start]['qsf']
for i in range(idx_start, len(df_on)):
    # solar_field_model(Tin: conHotTemperatureType, q: PositiveFloat, I: PositiveFloat, Tamb: float,
    #                   Tout_ant: float, q_ant: np.ndarray[float] = None,
    #                   beta: float = 0.0975, H: float = 2.2, nt=1, np=7 * 5, ns=2, Lt=1.15 * 20, Acs:float= 7.85e-5,
    #                   sample_time=1) -> float:
    ds = df_on.iloc[i]
    j=i-(idx_start-span)
    
    start_time = time.time()
    
    qsf_mod[j] =solar_field_inverse_model(
        Tin=df_on.iloc[i-span:i]['Tsf_in'].values, 
        Tout=ds['Tsf_out'], I=ds['I'], Tamb=ds['Tamb'],
        Tout_ant=df_on.iloc[i-1]['Tsf_out'],
        q_ant=qsf_mod[j:j-span:-1],
        sample_time=sample_rate_numeric, consider_transport_delay=True, 
        filter_signal=True, f=0.1,
        # Model parameters
        beta=model.beta_sf, H=model.H_sf, gamma=model.gamma_sf,
    )
    
    logger.info(f"Iteration {i} / {len(df_on)}, elapsed time: {time.time()-start_time:.2f} s")
    

# Remove the  span samples, since they are not really predictions from the model
# qsf_mod = qsf_mod[span:]

# Create a dataframe for both the original and filtered data
df_mod = pd.DataFrame({'qsf': qsf_mod}, index=df_on.index)

In [74]:
# Generate visualization (inverse model)

# Load plot configuration
with open(config_path / 'plt_config_solar_field.json') as f:
    plt_config = json.load(f)

# Sync model index with measured data
# df_mod.index = df_on.index[idx_start:i if i < idx_end - 1 else idx_end]
df_mod.index = df_on.index

fig = experimental_results_plot(plt_config, df_on, df_comp=[df_mod], vars_config=vars_config, resample=resample_figures)

# Save figure
save_figure(
    figure_name=f"{date_str}_solar_field_inverse_validation", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'),
    width=fig.layout.width, height=fig.layout.height, scale=2
)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'{date_str}_solar_field_inverse_validation')
# )

## Heat exchanger

In [75]:
# Evaluate model

from solarMED_modeling.heat_exchanger import heat_exchanger_model, calculate_heat_transfer_effectiveness

df["epsilon"] = np.nan

df_mod = pd.DataFrame()

# Run model
for idx in range(idx_start,idx_end):
    
    ds = df.iloc[idx]
        
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
    
    Thx_p_in = ds['Thx_p_in']
    Thx_s_in = ds['Thx_s_in']
    qhx_p = within_range_or_zero_or_max(ds['qhx_p'], range=[5, 10])
    # qhx_s = within_range_or_zero_or_max(ds['qhx_s'], range=[0.9, 1.7])
    qhx_s = ds['qhx_s']
    Tamb = ds['Tamb']
    
    Thx_p_out, Thx_s_out, epsilon = heat_exchanger_model(Tp_in=Thx_p_in, Ts_in=Thx_s_in, qp=qhx_p, qs=qhx_s, Tamb=Tamb, UA=model.UA_hx, H=model.H_hx, return_epsilon=True)
    
    try:
        estimated_epsilon = calculate_heat_transfer_effectiveness(Tp_in=Thx_p_in, Tp_out=ds["Thx_p_out"], Ts_in=Thx_s_in, Ts_out=ds["Thx_s_out"], qp=qhx_p, qs=qhx_s)
        
        df.loc[df.index[idx], 'epsilon'] = estimated_epsilon
    except:
        pass
    
    result = pd.DataFrame({
        'Thx_p_out': Thx_p_out,
        'Thx_s_out': Thx_s_out,
        'epsilon': epsilon,
        'qhx_s': qhx_s
    }, index=[0])
    

    logger.info(f"Finished iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} s")
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)    

In [76]:
# Generate visualization

# Load plot configuration
with open(config_path / 'plt_config_heat_exchanger.json') as f:
    plt_config = json.load(f)

# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=[df_mod], vars_config=vars_config, resample=resample_figures)

# Save figure
save_figure(
    figure_name=f"{date_str}_heat_exchanger_validation", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'),
    width=fig.layout.width, height=fig.layout.height, scale=2
)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'heat_exhanger_validation_{df.index[0].strftime("%Y%m%d")}')
# )

## MED

In [77]:
import MED_model
import matlab

MED_model_matlab = MED_model.initialize()
logger.info('MATLAB engine initialized')
    
idx_start = 0
idx_end = len(df)
df_mod = pd.DataFrame()

# Run model
for idx in range(idx_start,idx_end):
    ds = df.iloc[idx]
    start_time = time.time()

    Tmed_c_out = ds["Tmed_c_out"]
    Tmed_c_in = ds["Tmed_c_in"]
    mmed_f = ds["qmed_f"] if abs(ds["qmed_f"]-5) > 0.5 else 5
    mmed_f = within_range_or_nan_or_max(mmed_f, range=(5, 9), var_name="qmed_f")
    mmed_s = within_range_or_zero_or_max(ds["qmed_s"], range=(30, 48), var_name="qmed_s")
    Tmed_s_in = within_range_or_nan_or_max(ds["Tmed_s_in"], range=(60, 75), var_name="Tmed_s_in")

    MsIn = matlab.double([mmed_s / 3.6], size=(1, 1))  # m³/h -> L/s
    TsinIn = matlab.double([Tmed_s_in], size=(1, 1))
    MfIn = matlab.double([mmed_f], size=(1, 1))
    TcwinIn = matlab.double([Tmed_c_in], size=(1, 1))
    op_timeIn = matlab.double([0], size=(1, 1))
                
    med_model_solved = False
    mmed_d = np.nan
    Tmed_s_out = np.nan
    mmed_c = np.nan
    while not med_model_solved and (Tmed_c_in < Tmed_c_out < 40) and mmed_f > 0 and mmed_s > 0 and Tmed_s_in > 0 and Tmed_s_in > 0:   
        # try:                     
        TcwoutIn = matlab.double([Tmed_c_out], size=(1, 1))
    
        mmed_d, Tmed_s_out, mmed_c, _, _ = MED_model_matlab.MED_model(
            MsIn,  # L/s
            TsinIn,  # ºC
            MfIn,  # m³/h
            TcwoutIn,  # ºC
            TcwinIn,  # ºC
            op_timeIn,  # hours
            nargout=5
        )

        if mmed_c > 20:
            Tmed_c_out = Tmed_c_out + 1
        elif mmed_c < 9:
            Tmed_c_out = Tmed_c_out - 1
        else:
            med_model_solved = True
            
    logger.info(f"Iteration {idx} / {len(df)}. Elapsed time: {time.time()-start_time:.2f} seconds")
    
    result = pd.DataFrame(
        {
            "qmed_f": mmed_f,
            "qmed_d": mmed_d,
            "qmed_c": mmed_c,
            "qmed_s": mmed_s,
            "Tmed_s_in": Tmed_s_in,
            "Tmed_s_out": Tmed_s_out,
            "Tmed_c_in": ds["Tmed_c_in"],
            "Tmed_c_out": Tmed_c_out,
        },
        index=[0]
    )
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)

In [78]:
# Load plot configuration
with open(config_path / 'plt_config_med.json') as f:
    plt_config = json.load(f)

# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

# Save figure
save_figure(
    figure_name=f"{date_str}_MED_validation", 
    figure_path=attachments_folder,
    fig=fig, formats=('svg', 'html'),
    width=fig.layout.width, height=fig.layout.height, scale=2
)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'{df.index[0].strftime("%Y%m%d")}_med_validation')
# )

In [ ]:
model_config = 

text = "".join([ f"{key}: {value}\n" for key, value in model.__dict__.items()])

## Build report

In [79]:
test_report = f"""

---
Generated at {datetime.datetime.now().strftime("%Y-%m-%d %H:%M")}
Sample rate: {sample_rate}
{''.join(['- '])}

---

# Solar MED model validation report for test {date_str}

Validation report for Solar MED model, it includes validation graphs for the individual [component models](#Components) as well as the [complete system](#Complete system).

For the different visualizations, a static version if shown here, but an interactive `html` version is also available that can be opened in any browser. The link to it is shown above the static image.

# Test visualization

[Interactive version](attachments/{date_str}_solarMED_visualization.html)
![Test representation](attachments/{date_str}_solarMED_visualization.svg)

# Components

## Solar field

More detailed information about the model can be found in the [model documentation](../../models/solar_field.md).

- Parameters

| Parameter | Value |
| --------- | ----- |
| β (m)     |  {model.beta_sf:.2e}     |
| H (W/m²)? |  {model.H_sf:.2f}     |

### Temperature prediction

- Performance metrics

| Metric | Value |
| ------ | ----- |
| IAE    |       |
| RMSE   |       |
| ITAE   |       |
| MAE    |       |


[Interactive version](attachments/{date_str}_solar_field_validation.html)
![Solar field validation](attachments/{date_str}_solar_field_validation.svg)

### Inverse (flow prediction)


- Performance metrics

| Metric | Value |
| ------ | ----- |
| IAE    |       |
| RMSE   |       |
| ITAE   |       |
| MAE    |       |

[Interactive version](attachments/{date_str}_solar_field_inverse_validation.html)
![Solar_field_inverse validation](attachments/{date_str}_solar_field_inverse_validation.svg)

## Heat exchanger

More detailed information about the model can be found in the [model documentation](../../models/heat_exchanger.md).

- Parameters

| Parameter | Value |
| --------- | ----- |
| UA (W/K)  | {model.UA_hx:.2e}      |
| H (W/m²)? | {model.H_hx:.2f}     |

- Performance metrics

| Metric | Value |
| ------ | ----- |
| IAE    |       |
| RMSE   |       |
| ITAE   |       |
| MAE    |       |

[Interactive version](attachments/{date_str}_heat_exchanger_validation.html)
![solar MED validation](attachments/{date_str}_heat_exchanger_validation.svg)

## Thermal storage

More detailed information about the model can be found in the [model documentation](../../models/thermal_storage.md).

- Parameters

| Parameter    | Top        | Medium     | Bottom     |
| ------------ | ---------- | ---------- | ---------- |
| $UA_h$ (W/K) | {model.UAts_h[0]:.2e} | {model.UAts_h[1]:.2e} | {model.UAts_h[2]:.2e} |
| $V_h$ (m³)   | {model.Vts_h[0]:.2f} | {model.Vts_h[1]:.2f} | {model.Vts_h[2]:.2f} |
| $UA_c$ (W/K) | {model.UAts_c[0]:.2e} | {model.UAts_c[1]:.2e} | {model.UAts_c[2]:.2e} |
| $V_c$ (m³)   | {model.Vts_c[0]:.2f} | {model.Vts_c[1]:.2f} | {model.Vts_c[2]:.2f} |

- Performance metrics

| Metric | Value |
| ------ | ----- |
| IAE    |       |
| RMSE   |       |
| ITAE   |       |
| MAE    |       |

[Interactive version](attachments/{date_str}_thermal_storage_validation.html)
![Thermal storage validation](attachments/{date_str}_thermal_storage_validation.svg)

## MED

More detailed information about the model can be found in the [model documentation](../../models/MED.md).

- Parameters


- Performance metrics

| Metric | Value |
| ------ | ----- |
| IAE    |       |
| RMSE   |       |
| ITAE   |       |
| MAE    |       |

[Interactive version](attachments/{date_str}_MED_validation.html)
![MED validation](attachments/{date_str}_MED_validation.svg)

# Complete system

More detailed information about the model can be found in the [model documentation](../../models/complete_system.md).

- Parameters

(For some reason the static image generation is broken, but the interactive version is displayed correctly)

[Interactive version](attachments/{date_str}_solarMED_validation.html)
![solar MED validation](attachments/{date_str}_solarMED_validation.svg)

"""

## Build presentation report

In [ ]:
test_presentation = f"""

"""

In [80]:
# Save report

# _at_{datetime.datetime.now().strftime("%Y%m%dT%H%M")}
with open(output_folder / f'{date_str}_solarMED_validation_report.md', 'w') as f:
    f.write(test_report)
    
with open(output_folder / f'{date_str}_solarMED_validation_presentation.md', 'r') as f:
    f.write(test_presentation)